# Datasets

A Dataset is a specialization of a `Resource` that aims to register (upload) files with its metadata.

## Initialisation

Run the [Blue Brain Nexus project creation notebook](./00%20-%20Nexus_Project_Initialisation.ipynb) to create a Blue Brain Nexus project if you don't have one.

In [ ]:
!pip install git+https://github.com/BlueBrain/nexus-forge

In [ ]:
import getpass

The [Nexus web application](https://sandbox.bluebrainnexus.io/web) can be used to login and get a token.

- Step 1: From the opened web page, click on the login button on the right corner and follow the instructions.

![login-ui](https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/login-ui.png)

- Step 2: At the end you’ll see a token button on the right corner. Click on it to copy the token.

![login-ui](https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/copy-token.png)


In [ ]:
token = getpass.getpass()

In [ ]:
# Clone the repository if in Google Colab
import os 

!pwd
tutorial_base_dir = "./nexus-forge"
if os.path.exists(tutorial_base_dir):
  !rm -Rf $tutorial_base_dir

!git clone --single-branch https://github.com/BlueBrain/nexus-forge.git


os.chdir("/".join([tutorial_base_dir,"examples/notebooks/nexus-demo"]))

print("The working directory is now:")
!pwd

In [ ]:
#Let get some SHACL shapes from https://github.com/INCF/neuroshapes.git
import os 

neuroshapes_dir = "./neuroshapes"
if os.path.exists(neuroshapes_dir):
  !rm -Rf $neuroshapes_dir
! git clone https://github.com/INCF/neuroshapes.git
! cp -R "./neuroshapes/shapes/neurosciencegraph/datashapes/core/dataset" "./neuroshapes/shapes/neurosciencegraph/commons/" 
! cp -R "./neuroshapes/shapes/neurosciencegraph/datashapes/core/activity" "./neuroshapes/shapes/neurosciencegraph/commons/" 
! cp -R "./neuroshapes/shapes/neurosciencegraph/datashapes/core/entity" "./neuroshapes/shapes/neurosciencegraph/commons/" 
! cp -R "./neuroshapes/shapes/neurosciencegraph/datashapes/core/ontology" "./neuroshapes/shapes/neurosciencegraph/commons/" 
! cp -R "./neuroshapes/shapes/neurosciencegraph/datashapes/core/person" "./neuroshapes/shapes/neurosciencegraph/commons/" 

In [ ]:
# Set up some configurations

org ="tutorialnexus"
project ="myProject"
bucket = org+"/"+project
endpoint = "https://sandbox.bluebrainnexus.io/v1"


config = {
  "Model": {
    "name": "RdfModel",
    "origin": "directory",
    "source": "./neuroshapes/shapes/neurosciencegraph/commons/",
    "context": {
      "iri": "./neuroshapes_context.json"
    }
  },
  "Store": {
    "name": "BlueBrainNexus",
    "endpoint": "https://sandbox.bluebrainnexus.io/v1",
    "versioned_id_template": "{x.id}?rev={x._store_metadata._rev}",
    "file_resource_mapping": "../../configurations/nexus-store/file-to-resource-mapping.hjson"
  },
  "Formatters": {
    "identifier": "https://kg.example.ch/{}/{}"
  }
}


In [ ]:
from kgforge.core import KnowledgeGraphForge

In [ ]:
# Get a KnowledgeGraphForge session

forge = KnowledgeGraphForge(config, endpoint=endpoint,bucket=bucket, token=token)

## Imports

In [ ]:
from kgforge.core import Resource
from kgforge.specializations.resources import Dataset
import pandas as pd

## Creation with files

In [ ]:
! ls -p ../../data | egrep -v /$

In [ ]:
jane = Resource(type="Person", name="Jane Doe")

In [ ]:
persons = Dataset(forge, name="Interesting Persons")

In [ ]:
persons.add_files("../../data/persons.csv")

In [ ]:
persons.add_contribution(jane)

In [ ]:
forge.register(persons)

In [ ]:
forge.as_json(persons)

In [ ]:
associations = Dataset(forge, name="Associations data")

In [ ]:
associations.add_files("../../data/associations.tsv")

In [ ]:
associations.add_derivation(persons)

In [ ]:
associations.add_contribution(jane)

In [ ]:
forge.register(associations)

In [ ]:
forge.as_json(associations)


In [ ]:
associations.download("files", "./downloaded/")

In [ ]:
! ls ./downloaded

In [ ]:
! rm -R ./downloaded

## Creation with resources

In [ ]:
distribution_1 = forge.attach("../../data/associations.tsv")

In [ ]:
distribution_2 = forge.attach("../../data/persons.csv")

In [ ]:
jane = Resource(type="Person", name="Jane Doe", distribution=distribution_1)

In [ ]:
john = Resource(type="Person", name="John Smith", distribution=distribution_2)

In [ ]:
persons = [jane, john]

In [ ]:
forge.register(persons)

In [ ]:
dataset = Dataset(forge, name="Interesting people")

In [ ]:
dataset.add_parts(persons)

In [ ]:
print(dataset)

In [ ]:
forge.register(dataset)

In [ ]:
dataset.download("parts", "./downloaded/")

In [ ]:
! ls ./downloaded

In [ ]:
! rm -R ./downloaded

### specifiying custom content-type

In [ ]:
data_file = forge.attach("../../data/my_data.xwz", content_type="application/xwz")

In [ ]:
experiment = Resource(type="Experiment", name="generated data", distribution=data_file)

In [ ]:
forge.register(experiment)

In [ ]:
print(forge.as_json(experiment))

## Creation from a dataframe

See notebook `DataFrame IO.ipynb` for details on conversions of instances of Resource from a Pandas DataFrame.

### basics

In [ ]:
dataframe = pd.read_csv("../../data/persons.csv")

In [ ]:
dataframe

In [ ]:
persons = forge.from_dataframe(dataframe)

In [ ]:
forge.register(persons)

In [ ]:
dataset = Dataset(forge, name="Interesting people")

In [ ]:
dataset.add_parts(persons)

In [ ]:
print(dataset)

In [ ]:
forge.register(dataset)

### advanced

In [ ]:
dataframe = pd.read_csv("../../data/associations.tsv", sep="\t", usecols=["name", "type", "agent__type", "agent__name", "agent__gender__label", "distribution"])

In [ ]:
dataframe

In [ ]:
dataframe["distribution"] = dataframe["distribution"].map(lambda x: forge.attach(x))

In [ ]:
associations = forge.from_dataframe(dataframe, na="(missing)", nesting="__")

In [ ]:
print(associations[0])

In [ ]:
forge.register(associations)

In [ ]:
associations[0]._last_action

In [ ]:
associations[0]._synchronized

In [ ]:
associations[0]._store_metadata

In [ ]:
forge.as_json(associations[0])

In [ ]:
dataset = Dataset(forge, name="Interesting associations")

In [ ]:
print(dataset)

In [ ]:
dataset.add_parts(associations)

In [ ]:
print(dataset)

In [ ]:
forge.register(dataset)

In [ ]:
dataset.download("parts", "./downloaded/")

In [ ]:
! ls ./downloaded

In [ ]:
! rm -R ./downloaded